In [1]:
!pip install transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 32.3 MB/s 
     |████████████████████████████████| 441 kB 62.9 MB/s 
     |████████████████████████████████| 7.6 MB 54.3 MB/s 
     |████████████████████████████████| 163 kB 77.2 MB/s 
     |████████████████████████████████| 115 kB 57.5 MB/s 
     |████████████████████████████████| 212 kB 55.2 MB/s 
     |████████████████████████████████| 95 kB 4.5 MB/s 
     |████████████████████████████████| 127 kB 19.8 MB/s 
     |████████████████████████████████| 115 kB 54.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0.3.6
    Uninstalling dill-0.3.6:
      Successfully uninstalled dill-0.3.6


## import the encoder

In [2]:
from google.colab import files
uploaded = files.upload()

Saving encoder.py to encoder.py


In [3]:
import torch
from torch import nn
import torch.nn.functional as F

from torch.utils.data import dataset, DataLoader
import numpy as np
import math
import matplotlib.pyplot as plt

import encoder

## Load data and tokenizer and check

In [4]:
from transformers import AutoTokenizer, DataCollatorWithPadding
from datasets import load_dataset

In [5]:
checkpoint = 'distilbert-base-cased'
tokenizer = AutoTokenizer.from_pretrained(checkpoint)


Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

In [6]:
raw_datasets = load_dataset("glue", "sst2")

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [7]:
raw_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx'],
        num_rows: 1821
    })
})

In [8]:
def tokenize_fn(batch):
    return tokenizer(batch['sentence'], truncation=True)

In [9]:
tokenized_datasets = raw_datasets.map(tokenize_fn, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/68 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [10]:
data_collator

DataCollatorWithPadding(tokenizer=PreTrainedTokenizerFast(name_or_path='distilbert-base-cased', vocab_size=28996, model_max_len=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='pt')

In [11]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['sentence', 'label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [12]:
tokenized_datasets = tokenized_datasets.remove_columns(["sentence", "idx"])
tokenized_datasets = tokenized_datasets.rename_column("label", "labels")


In [13]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 67349
    })
    validation: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 872
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1821
    })
})

In [24]:
train_loader = DataLoader(
    tokenized_datasets['train'],
    shuffle=True,
    batch_size=32,
    collate_fn=data_collator
)
test_loader = DataLoader(
    tokenized_datasets['test'],
    batch_size=32,
    collate_fn=data_collator
)
valid_loader = DataLoader(
    tokenized_datasets['validation'],
    batch_size=32,
    collate_fn=data_collator
)

In [15]:
for batch in train_loader:
  for k, v in batch.items():
    print("k:", k, "v.shapes: ", v.shape)
  break

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


k: labels v.shapes:  torch.Size([32])
k: input_ids v.shapes:  torch.Size([32, 46])
k: attention_mask v.shapes:  torch.Size([32, 46])


In [16]:
set(tokenized_datasets['train']['labels'])

{0, 1}

In [17]:
tokenizer.vocab_size

28996

In [18]:
tokenizer.max_model_input_sizes

{'distilbert-base-uncased': 512,
 'distilbert-base-uncased-distilled-squad': 512,
 'distilbert-base-cased': 512,
 'distilbert-base-cased-distilled-squad': 512,
 'distilbert-base-german-cased': 512,
 'distilbert-base-multilingual-cased': 512}

## Train the encoder model

In [19]:

model = encoder.Encoder(
    vocab_size=tokenizer.vocab_size,
    max_len=tokenizer.max_model_input_sizes[checkpoint],
    d_k=16,
    d_model=64,
    n_heads=4,
    n_layers=2,
    n_classes=2,
    dropout_prob=0.1,
)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model.to(device)

Encoder(
  (embedding): Embedding(28996, 64)
  (pos_encoding): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_blocks): Sequential(
    (0): TransformerBlock(
      (k_embed): Linear(in_features=64, out_features=64, bias=True)
      (q_embed): Linear(in_features=64, out_features=64, bias=True)
      (v_embed): Linear(in_features=64, out_features=64, bias=True)
      (ln1): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (ln2): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
      (mha): MultiHeadAttention(
        (key): Linear(in_features=64, out_features=64, bias=True)
        (query): Linear(in_features=64, out_features=64, bias=True)
        (value): Linear(in_features=64, out_features=64, bias=True)
        (fc): Linear(in_features=64, out_features=64, bias=True)
      )
      (ann): Sequential(
        (0): Linear(in_features=64, out_features=256, bias=True)
        (1): GELU(approximate=none)
        (2): Linear(in_features=256

In [20]:
loss_func = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())

In [21]:
from datetime import datetime

def train(model, criterion, optimizer, train_loader, valid_loader, epochs):
  train_losses = np.zeros(epochs)
  test_losses = np.zeros(epochs)

  for it in range(epochs):
    model.train()
    t0 = datetime.now()
    train_loss = 0
    n_train = 0
    for batch in train_loader:
      batch = {k: v.to(device) for k, v in batch.items()}
      optimizer.zero_grad()
      outputs = model(batch['input_ids'], batch['attention_mask'])
      loss = criterion(outputs, batch['labels'])
      loss.backward()
      optimizer.step()

      train_loss += loss.item()*batch['input_ids'].size(0) 
      n_train += batch['input_ids'].size(0)

    train_loss = train_loss/n_train

    model.eval()
    test_loss = 0
    n_test = 0
    for batch in valid_loader:
      batch = {k: v.to(device) for k, v in batch.items()}
      outputs = model(batch['input_ids'], batch['attention_mask'])
      loss = criterion(outputs, batch['labels'])

      test_loss += loss.item()*batch['input_ids'].size(0) 
      n_test += batch['input_ids'].size(0)

    test_loss = test_loss/n_test
    
    train_losses[it] = train_loss
    test_losses[it] = test_loss

    dt = datetime.now() - t0
    print(f'Epoch {it+1}/{epochs}, Train Loss: {train_loss:.4f}, \
    Test Loss: {test_loss:.4f}, duration: {dt}')
  
  return train_losses, test_losses

In [22]:
train_losses, test_losses = train(model, loss_func, optimizer, train_loader, valid_loader, epochs = 4)

Epoch 1/4, Train Loss: 0.5531,     Test Loss: 0.5098, duration: 0:00:34.512313
Epoch 2/4, Train Loss: 0.3849,     Test Loss: 0.4447, duration: 0:00:32.496920
Epoch 3/4, Train Loss: 0.3142,     Test Loss: 0.5087, duration: 0:00:27.064807
Epoch 4/4, Train Loss: 0.2736,     Test Loss: 0.4719, duration: 0:00:27.820502


In [26]:
model.eval()
n_correct = 0
n_total = 0

for batch in train_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(batch['input_ids'], batch['attention_mask'])
    _, predictions = torch.max(outputs, 1)

    n_correct += (predictions == batch['labels']).sum().item()
    n_total += batch['labels'].shape[0]

train_acc = n_correct / n_total

for batch in test_loader:
    batch = {k: v.to(device) for k, v in batch.items()}
    outputs = model(batch['input_ids'], batch['attention_mask'])
    _, predictions = torch.max(outputs, 1)

    n_correct += (predictions == batch['labels']).sum().item()
    n_total += batch['labels'].shape[0]

test_acc = n_correct / n_total
print(train_acc, test_acc)

0.9220181442931595 0.897744687003036
